In [897]:
import os
import pandas as pd
import numpy as np
import math

In [898]:
def IssuerCode(state=False):
    issuers = pd.read_csv('data/s&p500.csv',sep=';')
    issuer_filtered = []
    for code_stock in issuers.Ticker:
        # Check last transaction 2023
        # Have 200 transaction
        path = f'data/transactions/{code_stock}.csv'
        if os.path.exists(path):
            historical = pd.read_csv(path)
            last_date_row = historical.Date.iloc[len(historical)-1] # get last date transactions
            data_test = historical[-1000:]['Close'] # get data test
            var = np.var(data_test) # calculate variance from data test
            if (last_date_row[:4] == '2023' and len(historical) >= 200 and var > 0):
                issuer_filtered.append(code_stock)
    if state:
        print("Code of Stock Issuer: {}".format(issuer_filtered))
    else:
        return issuer_filtered

In [899]:
issuers = IssuerCode()

In [900]:
def get_signal(df_pred):
    arr_signal = {'action':[]}
    lot = 0
    for k in range(len(df_pred)):
        action = "HOLD"
        if k > 0:
            prev = df_pred['y_predict'][k-1]
            current = df_pred['y_predict'][k]
            if current < prev and lot == 0:
                action = "BUY"
                lot = current
            elif current > lot and lot != 0:
                lot = 0
                action = "SELL"
        arr_signal['action'].append(action)
    return arr_signal

In [901]:
def trading(signals, df_test, cash=1000):
    lot = 0
    cash_beli = 0
    #harga dari dataset = harga per lembar
    for k in range(len(signals)):
        action = signals[k]
        open_price = df_test['Open'].iloc[k]
        close_price = df_test['Close'].iloc[k]
        if action == "BUY" and lot == 0 and cash > (open_price*100):
            lot += math.floor(cash/(open_price*100))
            cash_beli = open_price * (lot*100)
            cash -= cash_beli
        elif action == "SELL" and lot > 0:
            cash += close_price * (lot*100)
            lot = 0

        #last rows
        #1000 dan masih hold, dijual
        if k == (len(signals)-1) and lot > 0:
            cash += cash_beli
    return cash

In [902]:
# lookbacks = [5,10,15,20,25]
# architecture = ['TFGRU', 'TFLSTM', 'TFGRULSTM', 'TFLSTMGRU']
lookbacks = [20]
architecture = ['TFGRU']

issuer_result = {'code':[],'lookback':[],'arch':[],'cash_forecast':[], 'cash_sr':[]}

path_result = 'results/targets/'
initial_cash = 1000000
for code in issuers:
    path = f'data/transactions/{code}.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        df_test = df[-1000:]
        
        for archi in architecture:
            for lookback in lookbacks:
                path_target = f'{path_result}{archi}_multivariate/{code}_{lookback}_test_dfy.csv'
                df_pred = pd.read_csv(path_target)
                
                arr_signal = pd.DataFrame(get_signal(df_pred))
                cash = trading(arr_signal['action'], df_test, initial_cash)
                issuer_result['code'].append(code)
                issuer_result['lookback'].append(lookback)
                issuer_result['arch'].append(archi)
                issuer_result['cash_forecast'].append(cash)        


In [903]:
for code in issuers:
    path = f'data/transactions/{code}.csv'
    if os.path.exists(path):
        df = pd.read_csv(path)
        df_test = df[-1000:]
        path_signal = f'results/support_resistance_signals/{code}_signals.csv'
        
        arr_signal = pd.read_csv(path_signal)
        cash = trading(arr_signal['action'], df_test, initial_cash)
        issuer_result['cash_sr'].append(cash) 

In [904]:
# all_result
pd.options.display.float_format = '{:,.2f}'.format
df_result = pd.DataFrame(issuer_result)
df_result['return_forecast'] = df_result['cash_forecast'] - initial_cash
df_result['return_sr'] = df_result['cash_sr'] - initial_cash
df_result['return_forecast_percent'] = df_result['return_forecast'] / initial_cash * 100
df_result['return_sr_percent'] = df_result['return_sr'] / initial_cash * 100
#diatas 0% sudah untung
print(df_result.to_string(index=False))

 code  lookback  arch  cash_forecast      cash_sr  return_forecast    return_sr  return_forecast_percent  return_sr_percent
 AAPL        20 TFGRU   3,567,229.93 1,144,020.01     2,567,229.93   144,020.01                   256.72              14.40
 MSFT        20 TFGRU   1,993,725.18 1,231,721.92       993,725.18   231,721.92                    99.37              23.17
 AMZN        20 TFGRU   1,851,690.72   797,061.66       851,690.72  -202,938.34                    85.17             -20.29
 NVDA        20 TFGRU   5,718,151.97 1,781,519.84     4,718,151.97   781,519.84                   471.82              78.15
GOOGL        20 TFGRU   2,929,150.33   626,781.71     1,929,150.33  -373,218.29                   192.92             -37.32
 TSLA        20 TFGRU  10,589,596.19   491,208.01     9,589,596.19  -508,791.99                   958.96             -50.88
 META        20 TFGRU   1,922,053.89 2,343,085.28       922,053.89 1,343,085.28                    92.21             134.31
 GOOG   

In [905]:
# sum_all_code_result
df_sums = df_result[['cash_forecast', 'cash_sr', 'return_forecast', 'return_sr', 'return_forecast_percent', 'return_sr_percent']].max()
print(df_sums.to_string(index=False))

10,589,596.19
 2,343,085.28
 9,589,596.19
 1,343,085.28
       958.96
       134.31
